In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [0]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [0]:
df = pd.read_excel('UPCTEXT.xlsx', sheet_name = 'Sheet1')

In [0]:
df.head(10)

,ID,DISPLAY,text,Sentiment Score,Sentiment group,Topics
0,2,CONTROL,The ad provided basic information that was not...,NaN,NaN,NaN
1,4,CONTROL,i think the ad was informative. i believe it i...,NaN,NaN,NaN
2,6,CONTROL,"I think the add is about target marketing, and...",NaN,NaN,NaN
3,13,CONTROL,I think it's a good ad that highlights a major...,NaN,NaN,NaN
4,17,CONTROL,recycling is good.it is useful and helpful.it ...,NaN,NaN,NaN
5,18,CONTROL,"I think it is a good idea to reuse plastic, ho...",NaN,NaN,NaN
6,20,CONTROL,I believe that the study was about recycling i...,NaN,NaN,NaN
7,21,CONTROL,survey is very good. this product is so good a...,NaN,NaN,NaN
8,23,CONTROL,I think it's a great idea but not a very attra...,NaN,NaN,NaN
9,27,CONTROL,The picture of the shoe seemed a little dark a...,NaN,NaN,NaN


In [0]:
df.isnull().sum()

ID                   0
DISPLAY              0
text                 2
Sentiment Score    303
Sentiment group    303
Topics             303
dtype: int64

In [0]:
#replace the two rows that has nothin in it with no comment 
#maybe we can just leave it as it is
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 6 columns):
ID                 303 non-null int64
DISPLAY            303 non-null object
text               301 non-null object
Sentiment Score    0 non-null float64
Sentiment group    0 non-null float64
Topics             0 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 14.3+ KB


In [0]:
#change into string
df['text'] = df['text'].astype('str')

In [0]:
#dropping 2 rows with ''
string = ['']
cleaned = df[~df['text'].isin(string)]
cleaned.head(90)

,ID,DISPLAY,text,Sentiment Score,Sentiment group,Topics
0,2,CONTROL,The ad provided basic information that was not...,NaN,NaN,NaN
1,4,CONTROL,i think the ad was informative. i believe it i...,NaN,NaN,NaN
2,6,CONTROL,"I think the add is about target marketing, and...",NaN,NaN,NaN
3,13,CONTROL,I think it's a good ad that highlights a major...,NaN,NaN,NaN
4,17,CONTROL,recycling is good.it is useful and helpful.it ...,NaN,NaN,NaN
...,...,...,...,...,...,...
85,258,CONTROL,I did not have an opinion about the ad. I did ...,NaN,NaN,NaN
86,262,CONTROL,How willing are people to do something about t...,NaN,NaN,NaN
87,264,CONTROL,"I think it's a great shoe, but the ad just see...",NaN,NaN,NaN
88,267,CONTROL,"It was very informative, but was not visually ...",NaN,NaN,NaN


make lower case

In [0]:
cleaned['text'] = cleaned['text'].apply(lambda x: x.lower())

remove puncuation

In [0]:
cleaned['text'] = cleaned['text'].str.replace('[^\w\s]','')
cleaned['text'].head(20)

0     the ad provided basic information that was not...
1     i think the ad was informative i believe it is...
2     i think the add is about target marketing and ...
3     i think its a good ad that highlights a major ...
4     recycling is goodit is useful and helpfulit is...
5     i think it is a good idea to reuse plastic how...
6     i believe that the study was about recycling i...
7     survey is very good this product is so good an...
8     i think its a great idea but not a very attrac...
9     the picture of the shoe seemed a little dark a...
10    50 characters about a running shoe made from r...
11    this product is very useful and good quality t...
12    the ad was fairly plain and not especially wel...
13    i appreciate any company that uses recycled ma...
14    i think the ad is really just trying too hard ...
15    i liked it the only recycled shoes i have seen...
16    honestly i thought the ad was positive and con...
17    there was not enough color in the ad  spic

remove stopwords

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [0]:
cleaned['text'] = cleaned['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [0]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')


def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [0]:
cleaned['lemmatize'] = cleaned['text'].apply(lambda x: lemmatize_sentence(x))

In [0]:
df = cleaned[['text', 'lemmatize']]
df.head(30)

,text,lemmatize
0,ad provided basic information extremely persua...,ad provide basic information extremely persuas...
1,think ad informative believe worthwhile cause ...,think ad informative believe worthwhile cause ...
2,think add target marketing certain topic indiv...,think add target marketing certain topic indiv...
3,think good ad highlights major issue world tod...,think good ad highlight major issue world toda...
4,recycling goodit useful helpfulit effective,recycle goodit useful helpfulit effective
5,think good idea reuse plastic however shoes ad...,think good idea reuse plastic however shoes ad...
6,believe study recycling items reusing make new...,believe study recycle item reuse make new sneaker
7,survey good product good honest,survey good product good honest
8,think great idea attractive product perhaps se...,think great idea attractive product perhaps se...
9,picture shoe seemed little dark andor sharp cl...,picture shoe seem little dark andor sharp clar...


#LDA

In [0]:
!pip install LDA

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

In [0]:
def selected_topics(model, vectorizer, top_n=20):
    data_topic = dict()
    for idx, topic in enumerate(model.components_):
        #print("Topic %d:" % (idx))
        #print (" ".join([vectorizer.get_feature_names()[i]
        #                for i in topic.argsort()[:-top_n - 1:-1]]))
        data_topic["Topic_"+str(idx)]= [(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]
    return data_topic

In [0]:
vectorizer_ = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
df_vectorized = vectorizer_.fit_transform(cleaned['lemmatize'])

In [0]:
lda_ = LatentDirichletAllocation(n_components=3, max_iter=5, learning_method='online',verbose=True)
df_lda = lda_.fit_transform(df_vectorized)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [0]:
topic_vector =  selected_topics(lda_, vectorizer_)

In [0]:
#df_topic_vector = pd.DataFrame(topic_vector)
#df_topic_vector.to_excel("cleaned_df_10_topic_vector.xlsx")

#textblob

In [0]:
from textblob import TextBlob

In [0]:
def detect_polarity(text):
  score = TextBlob(text).sentiment.polarity
  print("{:-<40} {}".format(text, str(score)))

In [0]:
cleaned['Sentiment Score'] = cleaned['lemmatize'].apply(lambda text: TextBlob(text).sentiment.polarity)

In [0]:
tb_predicted_value = list(cleaned['Sentiment Score'])
#converting sentiment values to numpy for easier usage

compvall = np.array(tb_predicted_value)

len(tb_predicted_value)

303

In [0]:
cleaned['Sentiment Score'] = compvall

In [0]:
cleaned.tail(10)

,ID,DISPLAY,text,Sentiment Score,Sentiment group,Topics,lemmatize
293,272,R2L,okay ad talks plastic waste making shoes belie...,0.266667,NaN,NaN,okay ad talk plastic waste making shoe believa...
294,277,R2L,ad simple elicit emotion also visually appealing,0.000000,NaN,NaN,ad simple elicit emotion also visually appeal
295,280,R2L,ad intriguing hope true would absolutely buy s...,0.275000,NaN,NaN,ad intrigue hope true would absolutely buy sho...
296,282,R2L,felt message positive recycling important prot...,0.105455,NaN,NaN,felt message positive recycle important protec...
297,285,R2L,think placement shoes bottles switched visuall...,0.350000,NaN,NaN,think placement shoe bottle switch visually sh...
298,289,R2L,learning new ways recycle used plastic importa...,0.434091,NaN,NaN,learn new way recycle use plastic important sh...
299,292,R2L,love concept ad clear however ugly needs desig...,-0.033333,NaN,NaN,love concept ad clear however ugly need design...
300,297,R2L,ad saw beginning survey poorly done clearly th...,-0.018333,NaN,NaN,ad saw begin survey poorly do clearly thought ...
301,298,R2L,think ad wordy nice learn information feel lik...,0.433333,NaN,NaN,think ad wordy nice learn information feel lik...
302,300,R2L,ad basic easy understand however basic looked ...,-0.044444,NaN,NaN,ad basic easy understand however basic look fa...


In [0]:
#Assigning score categories and logic
i = 0

tb_predicted_value = [ ] #empty series to hold our predicted values

# Creating function for calculating positive, negative and neutral
# More than 1 --> Positive, equal to 0 --> neutral and less than 0 --> Negative
while(i<len(cleaned)):
    if ((cleaned.iloc[i]['Sentiment Score'] > 0)):
        tb_predicted_value.append('positive')
        i = i+1
    elif ((cleaned.iloc[i]['Sentiment Score'] == 0)):
        tb_predicted_value.append('neutral')
        i = i+1
    elif ((cleaned.iloc[i]['Sentiment Score'] < 0)):
        tb_predicted_value.append('negative')
        i = i+1

In [0]:
cleaned['Sentiment group'] = tb_predicted_value

In [0]:
cleaned.tail(10)

,ID,DISPLAY,text,Sentiment Score,Sentiment group,Topics,lemmatize
293,272,R2L,okay ad talks plastic waste making shoes belie...,0.266667,positive,NaN,okay ad talk plastic waste making shoe believa...
294,277,R2L,ad simple elicit emotion also visually appealing,0.000000,neutral,NaN,ad simple elicit emotion also visually appeal
295,280,R2L,ad intriguing hope true would absolutely buy s...,0.275000,positive,NaN,ad intrigue hope true would absolutely buy sho...
296,282,R2L,felt message positive recycling important prot...,0.105455,positive,NaN,felt message positive recycle important protec...
297,285,R2L,think placement shoes bottles switched visuall...,0.350000,positive,NaN,think placement shoe bottle switch visually sh...
298,289,R2L,learning new ways recycle used plastic importa...,0.434091,positive,NaN,learn new way recycle use plastic important sh...
299,292,R2L,love concept ad clear however ugly needs desig...,-0.033333,negative,NaN,love concept ad clear however ugly need design...
300,297,R2L,ad saw beginning survey poorly done clearly th...,-0.018333,negative,NaN,ad saw begin survey poorly do clearly thought ...
301,298,R2L,think ad wordy nice learn information feel lik...,0.433333,positive,NaN,think ad wordy nice learn information feel lik...
302,300,R2L,ad basic easy understand however basic looked ...,-0.044444,negative,NaN,ad basic easy understand however basic look fa...


In [0]:
#drop other columns
sentiment = cleaned.drop(['Topics'], axis = 1)
sentiment.tail(10)

,ID,DISPLAY,text,Sentiment Score,Sentiment group,lemmatize
293,272,R2L,okay ad talks plastic waste making shoes belie...,0.266667,positive,okay ad talk plastic waste making shoe believa...
294,277,R2L,ad simple elicit emotion also visually appealing,0.000000,neutral,ad simple elicit emotion also visually appeal
295,280,R2L,ad intriguing hope true would absolutely buy s...,0.275000,positive,ad intrigue hope true would absolutely buy sho...
296,282,R2L,felt message positive recycling important prot...,0.105455,positive,felt message positive recycle important protec...
297,285,R2L,think placement shoes bottles switched visuall...,0.350000,positive,think placement shoe bottle switch visually sh...
298,289,R2L,learning new ways recycle used plastic importa...,0.434091,positive,learn new way recycle use plastic important sh...
299,292,R2L,love concept ad clear however ugly needs desig...,-0.033333,negative,love concept ad clear however ugly need design...
300,297,R2L,ad saw beginning survey poorly done clearly th...,-0.018333,negative,ad saw begin survey poorly do clearly thought ...
301,298,R2L,think ad wordy nice learn information feel lik...,0.433333,positive,think ad wordy nice learn information feel lik...
302,300,R2L,ad basic easy understand however basic looked ...,-0.044444,negative,ad basic easy understand however basic look fa...


In [0]:
#df_topic_vector = pd.DataFrame(topic_vector)
#df_topic_vector.to_excel("cleaned_df_10_topic_vector.xlsx")
df_sentiment_score = pd.DataFrame(sentiment)
df_sentiment_score.to_excel('df_sentiment_score.xlsx')

In [0]:
sentiment['Sentiment group'].unique()

array(['negative', 'positive', 'neutral'], dtype=object)

In [0]:
pos = sentiment[sentiment['Sentiment group'] == 'positive']
neg = sentiment[sentiment['Sentiment group'] == 'negative']
neu = sentiment[sentiment['Sentiment group'] == 'neutral']

In [0]:
neu.head()

,ID,DISPLAY,text,Sentiment Score,Sentiment group,lemmatize
18,55,CONTROL,product novel ie recycled polyester shoes ad c...,0.0,neutral,product novel ie recycle polyester shoe ad cou...
38,116,CONTROL,survey related shoe brands shoe made recycled ...,0.0,neutral,survey relate shoe brand shoe make recycled pl...
45,137,CONTROL,ad running shoe used recycled materials said u...,0.0,neutral,ad run shoe use recycled material say use recy...
65,198,CONTROL,interesting idea picture falling apart dont se...,0.0,neutral,interest idea picture fall apart dont see coul...
69,210,CONTROL,liked idea ad thought copy ad could appealing ...,0.0,neutral,like idea ad think copy ad could appeal drive ...


In [0]:
pos_count = pos.shape[0]
neg_count = len(neg)
neu_count =  neu.shape[0]

In [0]:
pos_count

217

In [0]:
neg_count

50

In [0]:
neu_count

36

In [0]:
pos['DISPLAY'].unique()

array(['CONTROL', 'L2R', 'R2L'], dtype=object)

In [0]:
control_pos = pos.groupby('DISPLAY').count()
control_pos

,ID,text,Sentiment Score,Sentiment group,lemmatize
DISPLAY,,,,,
CONTROL,72,72,72,72,72
L2R,75,75,75,75,75
R2L,70,70,70,70,70


In [0]:
count_neg = neg.groupby('DISPLAY').count()
count_neg

,ID,text,Sentiment Score,Sentiment group,lemmatize
DISPLAY,,,,,
CONTROL,15,15,15,15,15
L2R,16,16,16,16,16
R2L,19,19,19,19,19


In [0]:
count_neu = neu.groupby('DISPLAY').count()
count_neu

,ID,text,Sentiment Score,Sentiment group,lemmatize
DISPLAY,,,,,
CONTROL,14,14,14,14,14
L2R,10,10,10,10,10
R2L,12,12,12,12,12


In [0]:
sentiment.groupby('DISPLAY').mean()

,ID,Sentiment Score
DISPLAY,,
CONTROL,152.108911,0.226141
L2R,153.009901,0.252900
R2L,150.881188,0.220921


In [0]:
vectorizer_ = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
pos_vectorized = vectorizer_.fit_transform(pos['lemmatize'])
lda_ = LatentDirichletAllocation(n_components=3, max_iter=5, learning_method='online',verbose=True)
pos_lda = lda_.fit_transform(pos_vectorized)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [0]:
topic_vector =  selected_topics(lda_, vectorizer_)

In [0]:
pos_topic_vector = pd.DataFrame(topic_vector)
pos_topic_vector.to_excel("pos_3topic_vector.xlsx")

In [0]:
vectorizer_ = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
neg_vectorized = vectorizer_.fit_transform(neg['lemmatize'])
lda_ = LatentDirichletAllocation(n_components=2, max_iter=5, learning_method='online',verbose=True)
neg_lda = lda_.fit_transform(neg_vectorized)

iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5


In [0]:
topic_vector =  selected_topics(lda_, vectorizer_)

In [0]:
neg_topic_vector = pd.DataFrame(topic_vector)
neg_topic_vector.to_excel("neg_2topic_vector.xlsx")